## PAE Oil Life Percentage Reset Metrics 

### Updated to use new Trip Summary table (May 2020)

In [1]:
import os, re, sys, utilities
spark_home = os.environ.get('SPARK_HOME', None)

In [2]:
code = utilities.getSparkContext()
exec(code)

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.3.2.3.1.5.80-6
      /_/

Using Python version 3.6.8 (default, Dec 30 2018 01:22:34)
SparkSession available as 'spark'.


In [3]:
from pyspark_llap.sql.session import HiveWarehouseSession
from pyspark.sql import SparkSession

#spark = SparkSession.builder.appName("LLAP_Test").enableHiveSupport().config("spark.sql.hive.llap", "true").getOrCreate()
spark = SparkSession.builder.appName("Hive_ACID_warehouseconnector").enableHiveSupport().getOrCreate()
hive = HiveWarehouseSession.session(spark).build()

In [4]:
import time
import pandas as pd
import numpy as np
import datetime
#from numba import jit
import math
import scipy
import matplotlib.pyplot as plt

In [5]:
from pyspark.sql.functions import col, udf
import pyspark.sql.functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import Row

In [6]:
from pyspark.sql.types import DoubleType, FloatType, BooleanType, LongType, \
IntegerType, TimestampType, StringType, StructType, StructField, ArrayType

In [7]:
sqlContext.setConf("spark.sql.shuffle.partitions", "2500")
sqlContext.setConf("spark.sql.orc.enabled", "true")

In [8]:
sqlContext.setConf("spark.sql.orc.filterPushdown", "true")
sqlContext.setConf("spark.sql.orc.char.enabled", "true")
sqlContext.setConf("spark.sql.adaptive.enabled ", "true")
sqlContext.setConf("spark.sql.cbo.enabled", "true")
sqlContext.setConf("spark.sql.orc.impl","native")

In [9]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 3000)

In [10]:
def oilDropReasonableDueToOdo(oil_drop_series,odo_increase_series):
    if (oil_drop_series <  odo_increase_series/8000.0*100.0*1.5 + 3).any():
        return True
    else:
        return False
    
def oilDropReasonableDueToTime(oil_drop_series,time_delta_series):
    if (oil_drop_series <  time_delta_series/86400.0/180*100*1.5 + 3).any():
        return True
    else:
        return False
    
def oilDropReasonable(oil_drop_series,odo_increase_series,time_delta_series):
    if oilDropReasonableDueToOdo(oil_drop_series,odo_increase_series) \
    or oilDropReasonableDueToTime(oil_drop_series, time_delta_series):
        return False
    else:
        return True  

In [11]:
def SuspectOilLife(df):
#     oilLifeChanged = df.oil.diff()
#     oilDrop = df.oil.diff() < 0
    oilBigDrop = df.oil.diff() < -1
    oilJump = df.oil.diff() > 0
    suspectOilDropOccurs = False
    suspectOilJumpOccurs = False
    oilJumpOccurs = False
    if oilBigDrop.any() == True:
        oil_drop_series = -df.oil.diff()[oilBigDrop]
        odo_increase_series = df.odo.diff()[oilBigDrop]
        time_delta_series = df.time_s.diff()[oilBigDrop].apply(func=lambda td:td.total_seconds())
        suspectOilDropOccurs = oilDropReasonable(oil_drop_series,odo_increase_series,time_delta_series)
    if oilJump.any() == True:
        oilJumpOccurs = True
        oil_jump_series = 100-df.oil[oilJump]
        odo_increase_series = df.odo.diff()[oilJump]
        time_delta_series = df.time_s.diff()[oilJump].apply(func=lambda td:td.total_seconds())
        suspectOilJumpOccurs = oilDropReasonable(oil_jump_series,odo_increase_series,time_delta_series)
        
    return suspectOilDropOccurs,suspectOilJumpOccurs,oilJumpOccurs
            

# Read trip summary data

In [12]:
#TODAY = datetime.datetime.strptime('2020-04-08', '%Y-%m-%d')
TODAY = datetime.datetime.today()

In [13]:
#changed start date to be YTD - Jan 01 2020
START_DATE_1 = "2020-01-01"
END_DATE_1 = (TODAY- datetime.timedelta(days=1)).strftime("%Y-%m-%d")
START_DATE_2 = (TODAY- datetime.timedelta(days=7)).strftime("%Y-%m-%d")
END_DATE_2 = (TODAY- datetime.timedelta(days=1)).strftime("%Y-%m-%d")
START_DATE_3 = (TODAY- datetime.timedelta(days=30)).strftime("%Y-%m-%d")
END_DATE_3 = (TODAY- datetime.timedelta(days=1)).strftime("%Y-%m-%d")

In [14]:
#df = hive.executeQuery("select * from sakula5_test.hello_acid")
#df.show()

trip_sum_allData = hive.executeQuery("""SELECT T1.cvdcqa_vin_d_3,
coalesce(T1.cvdcqa_oil_chng_cyc_no_r_3,0) as oil_cycle_no,
T1.cvdcqa_trip_no_r_3,
T1.cvdcqa_trip_end_time_s_3,
T1.cvdcqa_trip_end_eng_oil_p_3,
T1.cvdcqa_trip_end_odo_read_r_3,
T1.cvdcqa_partition_date,
T1.cvdcqa_partition_cntry_x
FROM cvdp.NCVDVCJ_TRIP_SUM_FINAL_4G_NA_USA_MSI_VW T1
WHERE T1.cvdcqa_trip_no_r_3 IS NOT NULL AND
T1.cvdcqa_trip_end_time_s_3 IS NOT NULL AND
T1.cvdcqa_trip_end_eng_oil_p_3 IS NOT NULL AND
T1.cvdcqa_trip_end_odo_read_r_3 IS NOT NULL AND
T1.cvdcqa_vin_d_3 IS NOT NULL AND
T1.cvdcqa_partition_cntry_x = 'USA' AND 
year(T1.cvdcqa_trip_end_time_s_3) > 2015""")
# """

In [15]:
#trip_sum_allData.show(5)

In [16]:
tripSumDf = trip_sum_allData\
.filter(col("cvdcqa_partition_date") >= START_DATE_1)\
.filter(col("cvdcqa_partition_date") <= END_DATE_1)\
.select(col("cvdcqa_trip_end_time_s_3").alias("time_s"),
    col("cvdcqa_trip_end_eng_oil_p_3").alias("oil"),
    col("cvdcqa_trip_end_odo_read_r_3").alias("odo"),
    col("cvdcqa_vin_d_3").alias("vin"))\
.filter(col('vin').isNotNull())\
.filter(col('odo').isNotNull())\
.filter(col('time_s').isNotNull())\
.filter((col('oil').isNotNull()))\
.withColumn('date',F.date_format("time_s","yyyy-MM-dd"))

In [17]:
#tripSumDf.show(5)

# Data since YTD (2020-01-01)

In [18]:
columnNames = tripSumDf.schema.names

def Combined(_row):    #Turns value a (a tuple) into a list of a single tuple.
    return [_row]

def MergeValue(_list, _row): #a is the new type [(,), (,), ..., (,)] and b is the old type (,)
    _list.append(_row)
    return _list

def MergeCombiners(_list1, _list2): #a is the new type [(,),...,(,)] and so is b, combine them
    _longlist = _list1 + _list2
    return _longlist

def Rename(dataf):
    dataf.columns = columnNames
    return dataf

def Reindex(dataf):
    dataf.index = range(0,len(dataf))
    return dataf

In [19]:
oilLifeTripSumPdRdd = tripSumDf.rdd.keyBy(lambda row:row.vin)\
.combineByKey(Combined,MergeValue,MergeCombiners,numPartitions=3000)\
.map(lambda tuple: pd.DataFrame(tuple[1])).map(Rename).map(lambda dataf: dataf.sort_values(['odo','time_s']))\
.map(Reindex)

### Ratio of suspected data 

In [20]:
suspectedCount = oilLifeTripSumPdRdd.map(SuspectOilLife).countByValue()
suspectedCount

defaultdict(int,
            {(False, False, True): 1628478,
             (False, False, False): 965644,
             (True, False, True): 3510,
             (False, True, True): 4505,
             (True, False, False): 401,
             (True, True, True): 53})

In [21]:
d_week = dict(suspectedCount)
float(d_week[(False, True, True)] + d_week[(True, False, True)] + d_week[(True, True, True)]) / \
(d_week[(False, False, False)] + d_week[(False, False, True)])

0.0031101081599092103

In [22]:
def oilResetOil(df):
    oilJump = df.oil.diff() > 0
    return df.oil.shift(1)[oilJump].as_matrix()

In [23]:
x = oilLifeTripSumPdRdd\
.filter(lambda df:SuspectOilLife(df)==(False, False, True))\
.flatMap(oilResetOil)\
.countByValue()

In [24]:
dictx = dict(x)
oilLifeList = []
countList = []
for k in dictx:
    oilLifeList.append(k)
    countList.append(dictx[k])
    
oilResetPd = pd.DataFrame({'oil':oilLifeList,'count':countList})

In [25]:
#sort ascending by oil % value
oilResetPd.sort_values(by=['oil'], inplace = True)

In [26]:
oilResetPd.to_csv("/s/lguerra5/oilReset"+START_DATE_1.replace("-","",2)+"_"+END_DATE_1.replace("-","",2)+".csv")

# Data of the last week

In [27]:
tripSumDf_week = trip_sum_allData\
.filter(col("cvdcqa_partition_date") >= START_DATE_2)\
.filter(col("cvdcqa_partition_date") <= END_DATE_2)\
.select(col("cvdcqa_trip_end_time_s_3").alias("time_s"),
    col("cvdcqa_trip_end_eng_oil_p_3").alias("oil"),
    col("cvdcqa_trip_end_odo_read_r_3").alias("odo"),
    col("cvdcqa_vin_d_3").alias("vin"))\
.filter(col('vin').isNotNull())\
.filter(col('odo').isNotNull())\
.filter(col('time_s').isNotNull())\
.filter((col('oil').isNotNull()))\
.withColumn('date',F.date_format("time_s","yyyy-MM-dd"))

In [28]:
oilLifeTripSumPdRdd_week = tripSumDf_week.rdd.keyBy(lambda row:row.vin)\
.combineByKey(Combined,MergeValue,MergeCombiners,numPartitions=3000)\
.map(lambda tuple: pd.DataFrame(tuple[1])).map(Rename).map(lambda dataf: dataf.sort_values(['odo','time_s']))\
.map(Reindex)

### Ratio of suspected data 

In [29]:
suspectedCount_week = oilLifeTripSumPdRdd_week.map(SuspectOilLife).countByValue()
suspectedCount_week

defaultdict(int,
            {(False, False, False): 2303822,
             (False, False, True): 69037,
             (True, False, False): 657,
             (True, False, True): 244,
             (False, True, True): 290,
             (True, True, True): 12})

In [30]:
d_week = dict(suspectedCount_week)
float(d_week[(False, True, True)] + d_week[(True, False, True)] + d_week[(True, True, True)]) / \
(d_week[(False, False, False)] + d_week[(False, False, True)])

0.0002301021678911389

In [31]:
x_week = oilLifeTripSumPdRdd_week\
.filter(lambda df:SuspectOilLife(df)==(False, False, True))\
.flatMap(oilResetOil)\
.countByValue()

In [32]:
dictx = dict(x_week)
oilLifeList = []
countList = []
for k in dictx:
    oilLifeList.append(k)
    countList.append(dictx[k])
    
oilResetPd_week = pd.DataFrame({'oil':oilLifeList,'count':countList})

In [33]:
#sort ascending by oil % value
oilResetPd_week.sort_values(by=['oil'], inplace = True)

In [34]:
#oilResetPd_week.to_csv("/u/lguerra5/oilReset"+START_DATE_2.replace("-","",2)+"_"+END_DATE_2.replace("-","",2)+".csv")

In [35]:
oilResetPd_week.to_csv("/s/lguerra5/oilReset"+START_DATE_2.replace("-","",2)+"_"+END_DATE_2.replace("-","",2)+".csv")

# Data of the last 30 days

In [36]:
tripSumDf_month = trip_sum_allData\
.filter(col("cvdcqa_partition_date") >= START_DATE_3)\
.filter(col("cvdcqa_partition_date") <= END_DATE_3)\
.select(col("cvdcqa_trip_end_time_s_3").alias("time_s"),
    col("cvdcqa_trip_end_eng_oil_p_3").alias("oil"),
    col("cvdcqa_trip_end_odo_read_r_3").alias("odo"),
    col("cvdcqa_vin_d_3").alias("vin"))\
.filter(col('vin').isNotNull())\
.filter(col('odo').isNotNull())\
.filter(col('time_s').isNotNull())\
.filter((col('oil').isNotNull()))\
.withColumn('date',F.date_format("time_s","yyyy-MM-dd"))

In [37]:
oilLifeTripSumPdRdd_month = tripSumDf_month.rdd.keyBy(lambda row:row.vin)\
.combineByKey(Combined,MergeValue,MergeCombiners,numPartitions=3000)\
.map(lambda tuple: pd.DataFrame(tuple[1])).map(Rename).map(lambda dataf: dataf.sort_values(['odo','time_s']))\
.map(Reindex)

### Ratio of suspected data 

In [38]:
suspectedCount_month = oilLifeTripSumPdRdd_month.map(SuspectOilLife).countByValue()
suspectedCount_month

defaultdict(int,
            {(False, False, False): 2142163,
             (False, False, True): 318757,
             (False, True, True): 1864,
             (True, False, True): 1503,
             (True, False, False): 868,
             (True, True, True): 61})

In [39]:
d_month = dict(suspectedCount_month)
float(d_month[(False, True, True)] + d_month[(True, False, True)] + d_month[(True, True, True)]) / \
(d_month[(False, False, False)] + d_month[(False, False, True)])

0.0013929749849649724

In [40]:
x_month = oilLifeTripSumPdRdd_month\
.filter(lambda df:SuspectOilLife(df)==(False, False, True))\
.flatMap(oilResetOil)\
.countByValue()
dictx = dict(x_month)
oilLifeList = []
countList = []
for k in dictx:
    oilLifeList.append(k)
    countList.append(dictx[k])
    
oilResetPd_month = pd.DataFrame({'oil':oilLifeList,'count':countList})

In [41]:
#sort ascending by oil % value
oilResetPd_month.sort_values(by=['oil'], inplace = True)

In [42]:
oilResetPd_month.to_csv("/s/lguerra5/oilReset"+START_DATE_3.replace("-","",2)+"_"+END_DATE_3.replace("-","",2)+".csv")

In [43]:
#oilResetPd_month.to_csv("/u/lguerra5/oilReset"+START_DATE_3.replace("-","",2)+"_"+END_DATE_3.replace("-","",2)+".csv")